<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 4
<br>
<b>Deadline:</b> March 30, 2020 (Monday). 23:00
</div>

# Exercise 5. Sequence-to-sequence modeling with recurrent neural networks

The goals of this exercise are
* to get familiar with recurrent neural networks used for sequential data processing
* to get familiar with the sequence-to-sequence model for machine translation
* to learn PyTorch tools for batch processing of sequences with varying lengths
* to learn how to write a custom `DataLoader`

You may find it useful to look at this tutorial:
* [Translation with a Sequence to Sequence Network and Attention](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [1]:
skip_training = False  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Data

The dataset that we are going to use consists of pairs of sentences in French and English.

In [7]:
from data import TranslationDataset, MAX_LENGTH, SOS_token, EOS_token

trainset = TranslationDataset(data_dir, train=True)

* `TranslationDataset` supports indexing as required by `torch.utils.data.Dataset`.
* Sentences are tensors of maximum length `MAX_LENGTH`.
* Words in a (sentence) tensor are represented as an index (integer) in a language vocabulary.
* The string representation of a word from the source language can be obtained from index `i` with `dataset.input_lang.index2word[i]`.
* Similarly for the target language `dataset.output_lang.index2word[j]`.

Let us look at samples from that dataset.

In [8]:
src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
print('Source sentence: "%s"' % ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
print('Sentence as tensor of word indices:')
print(src_sentence)

print('Target sentence: "%s"' % ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
print('Sentence as tensor of word indices:')
print(tgt_sentence)

Source sentence: "je t en supplie . EOS"
Sentence as tensor of word indices:
tensor([   6,  218,   14, 1143,    5,    1])
Target sentence: "i m begging you . EOS"
Sentence as tensor of word indices:
tensor([  2,   3, 620, 130,   4,   1])


In [9]:
print('Number of source-target pairs in the training set: ', len(trainset))

Number of source-target pairs in the training set:  8682


## Sequence-to-sequence model for machine translation

In this exercise, we are going to build a machine translation system which transforms a sentence in one language into a sentence in another one. The computational graph of the translation model is shown below:

<img src="seq2seq.png" width=900>

We are going to use a simplified model without the dotted connections.

## Custom DataLoader

We would like to train the sequence-to-sequence model using mini-batch training.
One difficulty of mini-batch training in this case is that sequences may have varying lengths and this has to be taken into account when building the computational graph. Luckily, PyTorch has tools to support batch processing of such sequences.
To use those tools, we need to write a custom data loader which puts sequences of varying lengths in the same tensor. We can customize the data loader by providing a custom `collate_fn` as explained [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

Our collate function:
- combines sequences from the source language in a single tensor with extra values (at the end) filled with `padding_value=0`.
- combines sequences from the target language in a single tensor with extra values (at the end) filled with `padding_value=0`.

**Important**:
- Late in the code (not in this `collate` function), we will convert source sequences to objects of class [`PackedSequence`](https://pytorch.org/docs/stable/nn.html?highlight=packedsequence#torch.nn.utils.rnn.PackedSequence) which can be processed by recurrent units such as `GRU` or `LSTM`. `PackedSequence` requires sequences to be sorted by their lengths.
**Therefore, the returned source sequences should be sorted by length in a decreasing order.**
* The target sequences need not be sorted by their lengths because we have to keep the same order of sequences in the source and target tensors.

Your task is to implement the collate function.

In [10]:
padding_value = 0

In [11]:
from torch.nn.utils.rnn import pad_sequence

def collate(list_of_samples):
    """Merges a list of samples to form a mini-batch.

    Args:
      list_of_samples is a list of tuples (src_seq, tgt_seq):
          src_seq is of shape (src_seq_length,)
          tgt_seq is of shape (tgt_seq_length,)

    Returns:
      src_seqs of shape (max_src_seq_length, batch_size): Tensor of padded source sequences.
          The sequences should be sorted by length in a decreasing order, that is src_seqs[:,0] should be
          the longest sequence, and src_seqs[:,-1] should be the shortest.
      src_seq_lengths: List of lengths of source sequences.
      tgt_seqs of shape (max_tgt_seq_length, batch_size): Tensor of padded target sequences.
    """
    src_list=[];
    tgt_list=[];
    src_seq_len=[];
    for i in list_of_samples:
        src=i[0]
        tgt=i[1]
        src_seq_len.append(len(src))
        src_list.append(src)
        tgt_list.append(tgt)
    src_list=sorted(src_list, key=len,reverse=True) 
    #src_list.sort(key=lambda x: len(x[0]))
    #src_list.sort(key=lambda t: len(t[1]), reverse=True)
    src_seqs=pad_sequence(src_list, batch_first=False, padding_value=0)
    tgt_seq=pad_sequence(tgt_list, batch_first=False, padding_value=0)
    return src_seqs, src_seq_len, tgt_seq

In [13]:
def test_collate_shapes():
    pairs = [
        (torch.LongTensor([1, 2]), torch.LongTensor([3, 4, 5])),
        (torch.LongTensor([6, 7, 8]), torch.LongTensor([9, 10])),
    ]
    pad_src_seqs, src_seq_lengths, pad_tgt_seqs = collate(pairs)
    assert pad_src_seqs.shape == torch.Size([3, 2]), f"Bad pad_src_seqs.shape: {pad_src_seqs.shape}"
    assert pad_src_seqs.dtype == torch.long
    assert pad_tgt_seqs.shape == torch.Size([3, 2]), f"Bad pad_tgt_seqs.shape: {pad_tgt_seqs.shape}"
    assert pad_tgt_seqs.dtype == torch.long
    print('Success')

test_collate_shapes()

Success


In [14]:
# This cell tests collate() function

In [15]:
# We create custom DataLoader using the implemented collate function
# We are going to process 64 sequences at the same time (batch_size=64)
from torch.utils.data import DataLoader
trainloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True, collate_fn=collate, pin_memory=True)

## Encoder

The encoder encodes a source sequence $(x_1, x_2, ..., x_T)$ into a single vector $h_T$ using the following recursion:
$$
  h_{t} = f(h_{t-1}, x_t) \qquad t = 1, \ldots, T
$$
where:
* intial state $h_0$ is often chosen arbitrarily (we choose it to be zero)
* function $f$ is defined by the type of the RNN cell (in our experiments, we will use [GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU))
* $x_t$ is a vector that represents the $t$-th word in the source sentence.

A common practice in natural language processing is to _learn_ the word representations $x_t$ (instead of, for example, using one-hot coded vectors). In PyTorch, this is supported by class [Embedding](https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding) which we are going to use.

The computational graph of the encoder is shown below:

<img src="seq2seq_encoder.png" width=500>

Your task is to implement the `forward` function of the encoder. It should contain the following steps:
* Embed the words of the source sequences.
* Pack source sequences using [`pack_padded_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pack_padded_sequence#torch.nn.utils.rnn.pack_padded_sequence). This converts padded source sequences into an object that can be processed by PyTorch recurrent units such as `nn.GRU` or `nn.LSTM`.
* Apply GRU computations to packed sequences obtained in the previous step
* Convert packed sequence of GRU outputs into padded representation with [`pad_packed_sequence`](https://pytorch.org/docs/stable/nn.html?highlight=pad_packed_sequence#torch.nn.utils.rnn.pad_packed_sequence).

In [16]:
class Encoder(nn.Module):
    def __init__(self, src_dictionary_size, embed_size, hidden_size):
        """
        Args:
          src_dictionary_size: The number of words in the source dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state of GRU.
        """
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(src_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)

    def forward(self, pad_seqs, seq_lengths, hidden):
        """
        Args:
          pad_seqs of shape (max_seq_length, batch_size): Padded source sequences.
          seq_lengths: List of sequence lengths.
          hidden of shape (1, batch_size, hidden_size): Initial states of the GRU.

        Returns:
          outputs of shape (max_seq_length, batch_size, hidden_size): Padded outputs of GRU at every step.
          hidden of shape (1, batch_size, hidden_size): Updated states of the GRU.
        """
        # YOUR CODE HERE
        x=self.embedding(pad_seqs)
        x=pack_padded_sequence(x,seq_lengths,batch_first=False)
        outputs, new_hidden = self.gru(x, hidden)
        
        outputs, _ = pad_packed_sequence(outputs, batch_first=False)
        return outputs, new_hidden
        #raise NotImplementedError()

    def init_hidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size)

In [17]:
def test_Encoder_shapes():
    hidden_size = 3
    encoder = Encoder(src_dictionary_size=5, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    hidden = encoder.init_hidden(batch_size=batch_size)
    pad_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])

    outputs, new_hidden = encoder.forward(pad_seqs=pad_seqs, seq_lengths=[4, 2], hidden=hidden)
    assert outputs.shape == torch.Size([4, batch_size, hidden_size]), f"Bad outputs.shape: {outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape: {new_hidden.shape}"
    print('Success')

test_Encoder_shapes()

Success


In [18]:
tests.test_Encoder(Encoder)

outputs[:, 0, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0221],
        [ 0.0007, -0.0055],
        [ 0.0005,  0.0323]])
outputs[:2, 1, :]:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
expected:
 tensor([[ 0.0000, -0.0150],
        [ 0.0004, -0.0021]])
new_hidden:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
expected:
 tensor([[[ 0.0005,  0.0323],
         [ 0.0004, -0.0021]]])
Success


## Decoder

The decoder takes as input the representation computed by the encoder and transforms it into a sentence in the target language. The computational graph of the decoder is shown below:

<img src="seq2seq_decoder.png" width=500 align="top">

* $z_0$ is the output of the encoder, that is $z_0 = h_5$, thus `hidden_size` of the decoder should be the same as `hidden_size` of the encoder.
* $y_{i}$ are the log-probabilities of the words in the target language, the dimensionality of $y_{i}$ is the size of the target dictionary.
* $z_{i}$ is mapped to $y_{i}$ using a linear layer `self.out` followed by `F.log_softmax` (because we use `nn.NLLLoss` loss for training).
* Each cell of the decoder is a GRU, it receives as inputs the previous state $z_{i-1}$ and relu of the **embedding** of the previous word. Thus, you need to embed the words of the target language as well. The previous word is taken as the word with the maximum log-probability.

Note that the decoder outputs a word at every step and the same word is used as the input to the recurrent unit at the next step. At the beginning of decoding, the previous word input is fed with a special word SOS which stands for "start of a sentence". During training, we know the target sentence for decoding, therefore we can feed the correct words $y_i$ as inputs to the recurrent unit.

There is one extra thing that it is wise to take care of. When the target sentence is fed to the decoder during training, the decoder learns to generate only the next word (this scenario is called "teacher forcing"). At test time, the decoder works differently: it generates the whole sequence using its own predictions as inputs at each step. Therefore, it makes sense to train the decoder to produce full sentences. In order to do that, we will alternate between two modes during training:
* "teacher forcing": the decoder is fed with the words in the target sequence
* no "teacher forcing": the decoder generates the output sequence using its own predictions. In this case, we will generate sequences of the same length as the length of the longest sequence in `pad_tgt_seqs` (if `pad_tgt_seqs` is not `None`) or of length `MAX_LENGTH` (if `pad_tgt_seqs` is `None`).

You need to implement the decoder which has the structure shown in the figure above.

Note:
* `SOS_token` is imported at the beginning of the notebook.

In [18]:
class Decoder(nn.Module):
    def __init__(self, tgt_dictionary_size, embed_size, hidden_size):
        """
        Args:
          tgt_dictionary_size: The number of words in the target dictionary.
          embed_size: The number of dimensions in the word embeddings.
          hidden_size: The number of features in the hidden state.
        """
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(tgt_dictionary_size, embed_size)
        self.gru = nn.GRU(input_size=embed_size, hidden_size=hidden_size)
        self.out = nn.Linear(hidden_size, tgt_dictionary_size)
        #self.tgt_dictionary_size = tgt_dictionary_size
        #self.softmax = nn.LogSoftmax(dim = 2)

    def forward(self, hidden, pad_tgt_seqs=None, teacher_forcing=False):
        """
        Args:
          hidden of shape (1, batch_size, hidden_size): States of the GRU.
          pad_tgt_seqs of shape (max_out_seq_length, batch_size): Tensor of words (word indices) of the
              target sentence. If None, the output sequence is generated by feeding the decoder's outputs
              (teacher_forcing has to be False).
          teacher_forcing (bool): Whether to use teacher forcing or not.

        Returns:
          outputs of shape (max_out_seq_length, batch_size, tgt_dictionary_size): Tensor of log-probabilities
              of words in the target language.
          hidden of shape (1, batch_size, hidden_size): New states of the GRU.

        Note: Do not forget to transfer tensors that you may want to create in this function to the device
        specified by `hidden.device`.
        """
        if pad_tgt_seqs is None:
            assert not teacher_forcing, 'Cannot use teacher forcing without a target sequence.'
            
        prev_word = torch.tensor(SOS_token * np.ones((1, hidden.size(1))), device = device, dtype = torch.int64)
        out_length = pad_tgt_seqs.size(0) if pad_tgt_seqs is not None else MAX_LENGTH
        outputs = []
        # YOUR CODE HERE
        for i in range(out_length):
            embedded = self.embedding(prev_word.view(1, -1))
            #print(embedded)
            #embedded = embedded.view(self.hidden_size)
            output = F.relu(embedded)
            output, hidden = self.gru(output, hidden)
            output = self.out(output)
            output= F.log_softmax(output, dim=2)
            outputs.append(output)
            #print(outputs.shape)
            if teacher_forcing:
                prev_word = pad_tgt_seqs[i]
                
            else: 
                prev_word =  torch.max(output, dim=2)[1].detach()
                # Use its own predictions as the next input
                #topv, topi = output[0, :].topk(1)
                #prev_word = topi.squeeze().detach()  # detach from history as input

                #if prev_word.item() == EOS_token:
                 #   break

        outputs = torch.cat(outputs, dim=0)  # [max_length, batch_size, output_dictionary_size]        
        return outputs, hidden
    
    #def init_hidden(self, batch_size=1):
        #return torch.zeros(1, batch_size, self.hidden_size)
        #output, hidden = self.gru()
        #raise NotImplementedError()

In [19]:
def test_Decoder_shapes():
    hidden_size = 2
    tgt_dictionary_size = 5
    test_decoder = Decoder(tgt_dictionary_size, embed_size=10, hidden_size=hidden_size)

    max_seq_length = 4
    batch_size = 2
    pad_tgt_seqs = torch.tensor([
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 0]
    ])  # [max_seq_length, batch_size]

    hidden = torch.zeros(1, batch_size, hidden_size)
    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=False)

    assert outputs.size(0) <= 4, f"Too long output sequence: outputs.size(0)={outputs.size(0)}"
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    outputs, new_hidden = test_decoder.forward(hidden, pad_tgt_seqs, teacher_forcing=True)
    assert outputs.shape == torch.Size([4, batch_size, tgt_dictionary_size]), \
        f"Bad shape outputs.shape={outputs.shape}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    # Generation mode
    outputs, new_hidden = test_decoder.forward(hidden, None, teacher_forcing=False)
    assert outputs.shape[1:] == torch.Size([batch_size, tgt_dictionary_size]), \
        f"Bad outputs.shape[1:]={outputs.shape[1:]}"
    assert new_hidden.shape == torch.Size([1, batch_size, hidden_size]), f"Bad new_hidden.shape={new_hidden.shape}"

    print('Success')

test_Decoder_shapes()

Success


In [20]:
tests.test_Decoder_no_forcing(Decoder)
tests.test_Decoder_with_forcing(Decoder)
tests.test_Decoder_generation(Decoder)

outputs[:, 0, :]:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
expected:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
outputs[:, 1, :]:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
expected:
 tensor([[-1.1366, -2.1924, -1.4361, -1.9640, -1.6645],
        [-1.3540, -1.8630, -1.5249, -1.7793, -1.6085],
        [-1.4899, -1.7024, -1.5838, -1.6901, -1.5962],
        [-1.5665, -1.6246, -1.6166, -1.6457, -1.5956]])
new_hidden:
 tensor([[[0.1003, 0.0421],
         [0.1003

## Training of sequence-to-sequence model using mini-batches

Now we are going to train the sequence-to-sequence model on the toy translation dataset.

In [21]:
# Create the seq2seq model
hidden_size = embed_size = 256
encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size).to(device)
decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size).to(device)

In [22]:
teacher_forcing_ratio = 0.5

Implement the training loop in the cell below. In the training loop, we first encode source sequences using the encoder, then we decode the encoded state using the decoder. The decoder outputs log-probabilities of words in the target language. We need to use these log-probabilities and the indexes of the words in the target sequences to compute the loss.

Recommended hyperparameters:
- Encoder optimizer: Adam with learning rate 0.001
- Decoder optimizer: Adam with learning rate 0.001
- Number of epochs: 30
- Toggle `teacher_forcing` on and off (for each mini-batch) according to the `teacher_forcing_ratio` specified above.

Hints:
- Training should proceed relatively fast.
- If you do well, the training loss should reach 0.1 in 30 epochs.
- **Important:** When computing the loss, you need to ignore the padded values. This can easily be done by using argument `ignore_index` of function [`nll_loss`](
https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.nll_loss).

In [23]:
if not skip_training:
    # YOUR CODE HERE
    encoder_optimizer = optim.Adam(encoder.parameters(), lr = 0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr = 0.001)
    num_epochs = 30
    #running_loss = 0.0
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_idx, batch in enumerate(trainloader):
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            pad_input_seqs, input_seq_lengths, pad_target_seqs = batch
            batch_size = pad_input_seqs.size(1)
            pad_input_seqs, pad_target_seqs = pad_input_seqs.to(device), pad_target_seqs.to(device)
            encoder_hidden = encoder.init_hidden(batch_size).to(device)
            _, encoder_hidden = encoder(pad_input_seqs, input_seq_lengths, encoder_hidden)            

            teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            decoder_outputs, decoder_hidden = decoder(encoder_hidden, pad_target_seqs, teacher_forcing=teacher_forcing)
            loss = 0
            for j in range(decoder_outputs.size(0)):
                loss += F.nll_loss(decoder_outputs[j], pad_target_seqs[j], ignore_index=0)
            loss = loss / decoder_outputs.size(0)
            loss.backward()

            encoder_optimizer.step()
            decoder_optimizer.step()    

            running_loss += loss.item()
            
            #if not epoch == 30:
                #print(f'Iteration {iteration} | Loss {loss.item():.2f}\n\n'))
   
            if not batch_idx % 32:
                print('Epoch: %03d/%03d | Batch %04d/%04d | Loss: %.4f' 
                   %(epoch+1, num_epochs, batch_idx+1, 
                     len(trainloader), running_loss/32))
                running_loss = 0.0

Epoch: 001/030 | Batch 0001/0136 | Loss: 0.2460
Epoch: 001/030 | Batch 0033/0136 | Loss: 4.1450
Epoch: 001/030 | Batch 0065/0136 | Loss: 2.9673
Epoch: 001/030 | Batch 0097/0136 | Loss: 2.7792
Epoch: 001/030 | Batch 0129/0136 | Loss: 2.6186
Epoch: 002/030 | Batch 0001/0136 | Loss: 0.0709
Epoch: 002/030 | Batch 0033/0136 | Loss: 2.3656
Epoch: 002/030 | Batch 0065/0136 | Loss: 2.2979
Epoch: 002/030 | Batch 0097/0136 | Loss: 2.2329
Epoch: 002/030 | Batch 0129/0136 | Loss: 2.2157
Epoch: 003/030 | Batch 0001/0136 | Loss: 0.0693
Epoch: 003/030 | Batch 0033/0136 | Loss: 2.0440
Epoch: 003/030 | Batch 0065/0136 | Loss: 2.1144
Epoch: 003/030 | Batch 0097/0136 | Loss: 2.0339
Epoch: 003/030 | Batch 0129/0136 | Loss: 1.9624
Epoch: 004/030 | Batch 0001/0136 | Loss: 0.0482
Epoch: 004/030 | Batch 0033/0136 | Loss: 1.8406
Epoch: 004/030 | Batch 0065/0136 | Loss: 1.8305
Epoch: 004/030 | Batch 0097/0136 | Loss: 1.7751
Epoch: 004/030 | Batch 0129/0136 | Loss: 1.8114
Epoch: 005/030 | Batch 0001/0136 | Loss:

In [24]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(encoder, '5_encoder.pth')
    tools.save_model(decoder, '5_decoder.pth')
else:
    hidden_size = 256
    encoder = Encoder(trainset.input_lang.n_words, embed_size, hidden_size)
    tools.load_model(encoder, '5_encoder.pth', device)
    
    decoder = Decoder(trainset.output_lang.n_words, embed_size, hidden_size)
    tools.load_model(decoder, '5_decoder.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 5_encoder.pth.
Do you want to save the model (type yes to confirm)? yes
Model saved to 5_decoder.pth.


In [25]:
# This cell tests training accuracy

## Evaluation

Next we need to implement a function that converts a source sequence to an output sequence using the trained sequence-to-sequence model.

In [28]:
def translate(encoder, decoder, src_seq):
    """Translate given sentence src_seq using trained encoder and decoder.
    
    Args:
      encoder (Encoder): Trained encoder.
      decoder (Decoder): Trained decoder.
      src_seq of shape (src_seq_length,): LongTensor of word indices of the source sequence.
    
    Returns:
      out_seq of shape (out_seq_length,): LongTensor of word indices of the output sequence.
    """
    with torch.no_grad():
        hidden = encoder.init_hidden().to(device)
        src_seq = src_seq.view(-1 , 1).to(device)
        output, hidden = encoder(src_seq, [src_seq.size(0)], hidden)
        output,hidden = decoder(hidden)
        decoder_outputs = torch.max(output, dim = 2)[1]
        
    return decoder_outputs
    #raise NotImplementedError()

In [29]:
def test_translate_shapes():
    src_seq = torch.tensor([1, 2, 3, 4]).to(device)
    out_seq = translate(encoder, decoder, src_seq)
    assert out_seq.shape[0] <= MAX_LENGTH, \
        f"Too long output sequence: tgt_seq.shape[0]={tgt_seq.shape[0]}"
    print('Success')

test_translate_shapes()

Success


Let us now translate random sentences from the training set and print the source, target, and produced output.

If you trained the model well enough, the model should memorize the training data well.

In [30]:
# Translate random sentences from the training set
print('Translate training data:')
print('-----------------------------')
for i in range(5):
    src_sentence, tgt_sentence = trainset[np.random.choice(len(trainset))]
    print('SRC:', ' '.join(trainset.input_lang.index2word[i.item()] for i in src_sentence))
    print('TGT:', ' '.join(trainset.output_lang.index2word[i.item()] for i in tgt_sentence))
    out_sentence = translate(encoder, decoder, src_sentence)
    print('OUT:', ' '.join(trainset.output_lang.index2word[i.item()] for i in out_sentence))
    print('')

Translate training data:
-----------------------------
SRC: elles sont furieuses apres toi . EOS
TGT: they re mad at you . EOS
OUT: they re mad at you . EOS EOS EOS EOS

SRC: je pars en turquie demain . EOS
TGT: i m off to turkey tomorrow . EOS
OUT: i m off to turkey tomorrow . EOS EOS EOS

SRC: ils sont nos invites . EOS
TGT: they are our guests . EOS
OUT: they are our guests . EOS EOS EOS EOS EOS

SRC: tu me decois vraiment . EOS
TGT: i m very disappointed in you . EOS
OUT: i m very disappointed in you . EOS EOS EOS

SRC: je viens de roumanie . EOS
TGT: i m from romania . EOS
OUT: i m from romania . EOS EOS EOS EOS EOS



Now we translate random sentences from the test set. A well-trained model should output sentences that look similar to the target ones. The mistakes are usually done for words that were rare in the training set.

In [31]:
testset = TranslationDataset(data_dir, train=False)

In [32]:
print('Translate test data:')
print('-----------------------------')
for i in range(5):
    src_sentence, tgt_sentence = testset[np.random.choice(len(testset))]
    print('SRC:', ' '.join(testset.input_lang.index2word[i.item()] for i in src_sentence))
    print('TGT:', ' '.join(testset.output_lang.index2word[i.item()] for i in tgt_sentence))
    out_sentence = translate(encoder, decoder, src_sentence)
    print('OUT:', ' '.join(testset.output_lang.index2word[i.item()] for i in out_sentence))
    print('')

Translate test data:
-----------------------------
SRC: ils mentent tous . EOS
TGT: they re all lying . EOS
OUT: they re all watching . EOS EOS EOS EOS EOS

SRC: j ai peur de mourir . EOS
TGT: i m afraid of death . EOS
OUT: i am afraid of dying . EOS EOS EOS EOS

SRC: il est nerveux d aller en amerique . EOS
TGT: he is anxious to go to america . EOS
OUT: he is nervous to the two team . EOS EOS

SRC: elle a arrete notre dispute . EOS
TGT: she stopped our fighting . EOS
OUT: she slipped into her . . EOS EOS EOS EOS

SRC: tu es vraiment magnifique . EOS
TGT: you re really gorgeous . EOS
OUT: you are really annoying . EOS EOS EOS EOS EOS

